In [82]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

# Otras
from tqdm import tqdm
import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random
import asyncio
import aiohttp
import re

import sys
import os
src_path = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path.append(src_path)
from support import obtener_sopa
from support import sacar_tabla


Creacion de Urls básicas de extraccion

In [5]:
url_facua = "https://super.facua.org/"

# Urls supermercados
supermercados=['mercadona','carrefour','eroski','dia','hipercor','alcampo']
urls_supermercados = [f"{url_facua}{keyword}/" for keyword in supermercados]

# Urls aceite girasol
producto='aceite-de-girasol'
urlsgirasol = [f"{url}{producto}/" for url in urls_supermercados]

# Urls aceite aceite de oliva
producto='aceite-de-oliva'
urlsoliva = [f"{url}{producto}/" for url in urls_supermercados]
tiposubproductoaceiteoliva=['suave-e-intenso','virgen','virgen-extra']
urlsolivasubtipos=[]
for producto2 in tiposubproductoaceiteoliva:
    urlsolivasubtipos.extend([f"{url}{producto2}/" for url in urlsoliva])

# Urls aceite aceite de oliva
producto='leche'
urlsleche = [f"{url}{producto}/" for url in urls_supermercados]
tiposubproductoleche=['enriquecida','entera-semi-desnatada','sin-lactosa']
urlslecheubtipos=[]
for producto2 in tiposubproductoleche:
    urlslecheubtipos.extend([f"{url}{producto2}/" for url in urlsleche])

# Juntar todas las urls en una lista
Listaurls=urlsgirasol+urlsolivasubtipos+urlslecheubtipos

In [6]:
Listaurls

['https://super.facua.org/mercadona/aceite-de-girasol/',
 'https://super.facua.org/carrefour/aceite-de-girasol/',
 'https://super.facua.org/eroski/aceite-de-girasol/',
 'https://super.facua.org/dia/aceite-de-girasol/',
 'https://super.facua.org/hipercor/aceite-de-girasol/',
 'https://super.facua.org/alcampo/aceite-de-girasol/',
 'https://super.facua.org/mercadona/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/carrefour/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/eroski/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/dia/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/hipercor/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/alcampo/aceite-de-oliva/suave-e-intenso/',
 'https://super.facua.org/mercadona/aceite-de-oliva/virgen/',
 'https://super.facua.org/carrefour/aceite-de-oliva/virgen/',
 'https://super.facua.org/eroski/aceite-de-oliva/virgen/',
 'https://super.facua.org/dia/aceite-de-oliva/virgen/',
 'https://super.f

Busqueda asincrona de Urls

In [7]:
# Función asíncrona para obtener el contenido de una URL y procesarlo con BeautifulSoup
async def search(url, session):
    async with session.get(url) as response:
        html = await response.text()
        return BeautifulSoup(html, "html.parser")  # Retorna la sopa de cada URL

# Función principal para manejar las peticiones asíncronas
async def main(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [search(url, session) for url in urls]
        results = await asyncio.gather(*tasks)
        return results
    # Ejecución de corutinas en entornos interactivos

listasopas = await main(Listaurls)
    


In [8]:
# Encuentra todos los enlaces que coincidan con el patrón
listatotalurlsproductos=[]

for sopa in listasopas:
    # Encuentra todas las etiquetas <a> con la clase 'btn-unirme'
    enlaces = sopa.find_all('a', class_=lambda c: c and 'btn-unirme' in c)
    
    # Extrae el atributo 'href' de cada enlace encontrado y agrégalo a la lista si existe
    for enlace in enlaces:
        href = enlace.get('href')
        if href and href.startswith('https://'):
            if href.endswith('unirme/'):
                None
            else:
                listatotalurlsproductos.append(href)
listatotalurlsproductos

['https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/',
 'https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-capicua-garrafa-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-carrefour-classic-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-carrefour-garrafa-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-coosol-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-especial-para-freir-capicua-3-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-fontasol-1-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-fontasol-5-l/',
 'https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-koipe-fritos-1-l/',
 'https://super.facua.org/carrefour/aceite-de-gi

In [12]:
listasopasproductos=[]
for url in listatotalurlsproductos:
    response = requests.get(url)
    # Comprobar que la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir el contenido de la página a BeautifulSoup
        sopa = BeautifulSoup(response.text, 'html.parser')
        listasopasproductos.append(sopa)
    else:
        print(f"Error al acceder a {url}: Código de estado {response.status_code}")

In [13]:
# Comprobar si todo esta sacado
print(len(listatotalurlsproductos))
print(len(listasopasproductos))

1531
1531


In [91]:
unasopa=listasopasproductos[0]

In [109]:
sopasdectuosas=[]
sopasbuenas=[]
for i in listasopasproductos:
    try:
        # Extraer el título del <h2>
        title = i.find('h2').text.strip('Tabla de precios por día para')

        info = [item.text.strip() for item in i.select('li.breadcrumb-item')][1:]

        # Extraer datos de la tabla
        data = []
        for fila in i.find_all('tr')[1:]:  # Omitir la fila de encabezados
            columnas = fila.find_all('td')
            dia = columnas[0].text.strip()
            precio = columnas[1].text.strip()
            variacion = columnas[2].text.strip()
            data.append([dia, precio, variacion])

        # Convertir en DataFrame de pandas
        df = pd.DataFrame(data, columns=["Día", "Precio (€)", "Variación"])

        # Agregar el título como una columna adicional
        df['Nombre'] = title
        df['Supermercado'] = info[0]
        df['Tipo producto'] = info[1]
        try:
            df['Tipo subproducto'] = info[2]
        except: None
        sopasbuenas.append(df)
    except:
        sopasdectuosas.append(i)

df_unido=pd.concat(sopasbuenas,ignore_index=True)


In [111]:
print(len(listasopasproductos))
print(len(sopasbuenas))
print(len(sopasdectuosas))

1531
1261
270


In [164]:
sopasbuenas2 = []
sopasdectuosas2 = []
titulosdefect=[]

for i in sopasdectuosas:
    try:
        # Extraer el título del <h2>
        title = i.find('h2').text.strip('Comparativa de precios de')
        titulosdefect.append(title)
        info = [item.text.strip() for item in i.select('li.breadcrumb-item')][1:]

        # # Extraer datos de la tabla
        tabla_precios = sopa.find_next("table")

        # Si la tabla está en divs en lugar de una tabla HTML
        if not tabla_precios:
            tabla_precios = sopa.find_next("div", class_="tabla-precios")  # Ajusta la clase si es necesario

        # Almacena los datos en listas y convierte a DataFrame
        if tabla_precios:
            filas = tabla_precios.find_all("tr")
            data = []
            for fila in filas:
                columnas = fila.find_all(["td", "th"])  # Considera tanto headers como datos
                datos = [columna.get_text(strip=True) for columna in columnas]
                if len(datos) == 3:  # Solo agrega filas que tengan las tres columnas Día, Precio, Variación
                    data.append(datos)

            # Crear el DataFrame con encabezados específicos
        df = pd.DataFrame(data, columns=["Día", "Precio (€)", "Variación"]).drop([0, 1]).reset_index(drop=True)
        # Agregar el título como una columna adicional
        df['Nombre'] = title
        df['Supermercado'] = info[0]
        df['Tipo producto'] = info[1]
        try:
            df['Tipo subproducto'] = info[2]
        except: None
        sopasbuenas2.append(df)
    
    except:
        sopasdectuosas2.append(i)

df_unido2=pd.concat(sopasbuenas2,ignore_index=True)


In [165]:
df_unido_final=pd.concat([df_unido,df_unido2],ignore_index=True)

In [167]:
df_unido_final

,Día,Precio (€),Variación,Nombre,Supermercado,Tipo producto
0,12/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
1,13/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
2,14/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
3,15/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
4,16/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
...,...,...,...,...,...,...
126453,21/10/2024,"9,15",=,"KAIKU Leche entera de vaca, sin lactosa 6 x 1l.",Alcampo,Leche
126454,22/10/2024,"9,15",=,"KAIKU Leche entera de vaca, sin lactosa 6 x 1l.",Alcampo,Leche
126455,23/10/2024,"9,15",=,"KAIKU Leche entera de vaca, sin lactosa 6 x 1l.",Alcampo,Leche
126456,24/10/2024,"9,15",=,"KAIKU Leche entera de vaca, sin lactosa 6 x 1l.",Alcampo,Leche


In [168]:
df_unido_final['Día'] = pd.to_datetime(df_unido_final['Día'], dayfirst=True)
df_unido_final['Precio (€)'] = df_unido_final['Precio (€)'].str.replace(',', '.').astype(float)


In [169]:
df_unido_final.dtypes

Día              datetime64[ns]
Precio (€)              float64
Variación                object
Nombre                   object
Supermercado             object
Tipo producto            object
dtype: object

In [170]:
df_unido.to_csv(f'../data/base/datosprecios.csv', index=False, encoding='utf-8')